In [ ]:
from IPython.core.display import HTML
HTML('''
<script>
var logoParent = document.getElementById("kernel_logo_widget")
var logo = document.getElementById("kernel_logo_widget").getElementsByClassName("current_kernel_logo")[0];
logo.src = "https://i.ibb.co/mD4jTGQ/itclogo.jpg";
logo.style = "display: inline; width:138px; height:40px";
logoParent.innerHTML = '<a href="https://i.ibb.co/mD4jTGQ/itclogo.jpg">' + logoParent.innerHTML + '</a>';
</script>
''')

<font size=10><b>Feature Selection - Exercise</b></font> 

In this exercise, we will use the adult_data_usa.csv data, that holds information about different people.
We will try to predict wheather a person earns more than 50K a year, or less than that.

We will use different feature selection techniques to select features for the models, and compare the recieved models.

# Questions

## load the dataset and explore it LIGHTLY

In [3]:
import pandas as pd
data = pd.read_csv("adult_data_usa.csv")
display(data.sample(5))
display(data.info())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,income
5229,73,Self-emp-not-inc,29306,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,40,<=50K
27989,34,Private,347166,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,<=50K
13782,36,Private,293528,Assoc-voc,11,Never-married,Tech-support,Not-in-family,White,Female,0,0,3,<=50K
12049,45,Private,162915,Some-college,10,Divorced,Transport-moving,Not-in-family,White,Male,0,0,50,<=50K
19456,33,Self-emp-not-inc,327902,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,45,<=50K


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29170 entries, 0 to 29169
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             29170 non-null  int64 
 1   workclass       29170 non-null  object
 2   fnlwgt          29170 non-null  int64 
 3   education       29170 non-null  object
 4   education-num   29170 non-null  int64 
 5   marital-status  29170 non-null  object
 6   occupation      29170 non-null  object
 7   relationship    29170 non-null  object
 8   race            29170 non-null  object
 9   sex             29170 non-null  object
 10  capital-gain    29170 non-null  int64 
 11  capital-loss    29170 non-null  int64 
 12  hours-per-week  29170 non-null  int64 
 13  income          29170 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.1+ MB


None

In [6]:
data['income'].value_counts()
data['workclass'].value_counts()

workclass
 Private             20135
 Self-emp-not-inc     2313
 Local-gov            1956
 ?                    1659
 State-gov            1210
 Self-emp-inc          991
 Federal-gov           886
 Without-pay            13
 Never-worked            7
Name: count, dtype: int64

## Randomly split the data into train and test sets (no need for validation at this point)

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, test_size=0.2, random_state=42)

## Encode all categorical features, and turn the dependent variable into a numeric feature as well

In [9]:
from sklearn.preprocessing import LabelEncoder

categorical_features = df_train.select_dtypes(include=['object']).columns
for feature in categorical_features:
    le = LabelEncoder()
    df_train[feature] = le.fit_transform(df_train[feature])
    df_test[feature] = le.transform(df_test[feature])

le = LabelEncoder()
df_train['income'] = le.fit_transform(df_train['income'])
df_test['income'] = le.transform(df_test['income'])
df_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,income
8426,35,4,225750,9,13,0,10,4,4,0,0,0,32,0
4121,43,4,191149,11,9,0,4,1,4,0,0,0,40,0
19220,32,4,210562,11,9,2,7,0,4,1,0,0,40,0
27110,20,4,117222,15,10,4,13,3,4,1,0,0,15,0
23205,62,6,162249,11,9,2,14,0,4,1,0,0,30,0


## Train a model of your choice, and evaluate it. What are your recall and precision?

In [12]:
# Let's use a RF classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

X_train = df_train.drop(columns=['income'])
y_train = df_train['income']
X_test = df_test.drop(columns=['income'])
y_test = df_test['income']

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)


Precision: 0.7353923205342237
Recall: 0.6156533892382949


## Try to select features with the Chi-square method, and one other filter method. Indicate which features were eliminated. Train the same model with the new subset of features and report your precision and recall

In [11]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif

selector_chi2 = SelectKBest(score_func=chi2, k=5)
X_train_chi2 = selector_chi2.fit_transform(X_train, y_train)
X_test_chi2 = selector_chi2.transform(X_test)
selected_features_chi2 = X_train.columns[selector_chi2.get_support()]

selector_f = SelectKBest(score_func=f_classif, k=5)
X_train_f = selector_f.fit_transform(X_train, y_train)
X_test_f = selector_f.transform(X_test)
selected_features_f = X_train.columns[selector_f.get_support()]

model_chi2 = RandomForestClassifier(random_state=42)
model_chi2.fit(X_train_chi2, y_train)
y_pred_chi2 = model_chi2.predict(X_test_chi2)

model_f = RandomForestClassifier(random_state=42)
model_f.fit(X_train_f, y_train)
y_pred_f = model_f.predict(X_test_f)

precision_chi2 = precision_score(y_test, y_pred_chi2)
recall_chi2 = recall_score(y_test, y_pred_chi2)

precision_f = precision_score(y_test, y_pred_f)
recall_f = recall_score(y_test, y_pred_f)

eliminated_features_chi2 = X_train.columns[~selector_chi2.get_support()]

eliminated_features_f = X_train.columns[~selector_f.get_support()]

print("Features eliminated by Chi-square method:", eliminated_features_chi2)
print("Features eliminated by ANOVA filter method:", eliminated_features_f)

print("Chi-square method:")
print("Precision:", precision_chi2)
print("Recall:", recall_chi2)
print("Selected features:", selected_features_chi2)

print("\nANOVA filter method:")
print("Precision:", precision_f)
print("Recall:", recall_f)
print("Selected features:", selected_features_f)


Features eliminated by Chi-square method: Index(['workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
       'occupation', 'race', 'sex'],
      dtype='object')
Features eliminated by ANOVA filter method: Index(['workclass', 'fnlwgt', 'education', 'marital-status', 'occupation',
       'race', 'sex', 'capital-loss'],
      dtype='object')
Chi-square method:
Precision: 0.6850618458610847
Recall: 0.5031446540880503
Selected features: Index(['age', 'relationship', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')

ANOVA filter method:
Precision: 0.6865046102263203
Recall: 0.5723270440251572
Selected features: Index(['age', 'education-num', 'relationship', 'capital-gain',
       'hours-per-week'],
      dtype='object')


## Try to select features with ONE wrapper method. Indicate which features were eliminated. Train the same model with the new subset of features and report your precision and recall

**Indicate how many features you kept, and PROVIDE CONCLUSIONS**

In [13]:
from sklearn.feature_selection import RFE

model = RandomForestClassifier(random_state=42)
rfe = RFE(model, n_features_to_select=5)
rfe.fit(X_train, y_train)
selected_features_rfe = X_train.columns[rfe.support_]
eliminated_features_rfe = X_train.columns[~rfe.support_]

model_rfe = RandomForestClassifier(random_state=42)
model_rfe.fit(X_train[selected_features_rfe], y_train)
y_pred_rfe = model_rfe.predict(X_test[selected_features_rfe])

precision_rfe = precision_score(y_test, y_pred_rfe)
recall_rfe = recall_score(y_test, y_pred_rfe)

print("Precision (RFE):", precision_rfe)
print("Recall (RFE):", recall_rfe)
print("Selected features (RFE):", selected_features_rfe)
print("Features eliminated (RFE):", eliminated_features_rfe)

Precision (RFE): 0.648062015503876
Recall (RFE): 0.5842068483577918
Selected features (RFE): Index(['age', 'fnlwgt', 'education-num', 'relationship', 'capital-gain'], dtype='object')
Features eliminated (RFE): Index(['workclass', 'education', 'marital-status', 'occupation', 'race', 'sex',
       'capital-loss', 'hours-per-week'],
      dtype='object')


## Please sumarize your results. Did things happened as you expected? Please elaborate.

We obtained different results for different of model of the same kind but with different selected features, here is the sumarized results : 

1. Selecting all the features :

   Precision: 0.7353923205342237

   Recall: 0.6156533892382949

2. Chi-square method :

    Precision: 0.6850618458610847

    Recall: 0.5031446540880503

    Selected features: ['age', 'relationship', 'capital-gain', 'capital-loss','hours-per-week']

3. ANOVA filter method :

   Precision: 0.6865046102263203

   Recall: 0.5723270440251572

   Selected features: ['age', 'education-num', 'relationship', 'capital-gain', 'hours-per-week']

4. ONE wrapper method :

   Precision : 0.648062015503876

   Recall : 0.5842068483577918

   Selected features : ['age', 'fnlwgt', 'education-num', 'relationship', 'capital-gain']



Overall, the results demonstrate variations in precision and recall across different feature selection methods. Notably, the model trained with all features achieves the highest precision and recall compared to the feature-selected models. The absence of a filter in feature selection enables the model to utilize all available information, resulting in better predictive performance and higher recall, while also potentially increasing the risk of overfitting. We can also note that when applying a certain filter of any kind, we obtain very similar results. Of course the first goal of applying a filter on a certain dataset is to improve its performance so at first sight, with this specific dataset, we would like to keep all the features 